# Single Cell apoptosis

As decribed in [Monier et al. 2015](http://www.nature.com/nature/journal/v518/n7538/full/nature14152.html).

We start with a simple hexagonal apical mesh with cylindrical symmetry. 

#### Imports

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt

import ipywidgets as widgets
from IPython.display import display, Image

import napari
import vispy as vp

import tyssue
from tyssue import Sheet, History
from tyssue import config

from tyssue import SheetGeometry as geom
from tyssue.dynamics.sheet_vertex_model import SheetModel as basemodel
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.solvers.quasistatic import QSSolver

from tyssue.config.draw import sheet_spec
from tyssue.utils.utils import spec_updater

from tyssue.draw.vispy_draw import sheet_view, face_visual, edge_visual
from tyssue.draw import browse_history
from tyssue.io.hdf5 import load_datasets


print(tyssue.__version__)

0.9.0


### Loading the datasets and specifications

Here it is a cylindrical hexagonal mesh.

In [2]:
# Read pre-recorded datasets
h5store = '../data/small_hexagonal.hf5'
from tyssue.io.hdf5 import save_datasets, load_datasets

datasets = load_datasets(h5store,
                         data_names=['face', 'vert', 'edge'])
# Corresponding specifications
specs = config.geometry.cylindrical_sheet()
sheet = Sheet('emin', datasets, specs)
sheet.sanitize(trim_borders=True, order_edges=True)

geom.update_all(sheet)
# Model
nondim_specs = config.dynamics.quasistatic_sheet_spec()
dim_model_specs = model.dimensionalize(nondim_specs)
sheet.update_specs(dim_model_specs)

sheet.get_opposite()
live_edges = sheet.edge_df[sheet.edge_df['opposite']==-1].index
dead_src = sheet.edge_df.loc[live_edges, 'srce'].unique()

### Boundary conditions
sheet.vert_df.is_active = 1
sheet.vert_df.loc[dead_src, 'is_active'] = 0

sheet.edge_df['is_active'] = sheet.upcast_srce('is_active') * sheet.upcast_trgt('is_active')

### First energy minimization

In [3]:

min_settings = {
#    "minimize":{
        'options': {
            'disp': False,
            'ftol': 1e-6,
            'gtol': 1e-5},
#    }
}
solver = QSSolver()

res = solver.find_energy_min(sheet, geom, model, **min_settings)
print(res['success'])

True


### Custom display function

In [4]:
def leg_joint_view(sheet, coords=['z', 'x', 'y']):
    
    geom.update_all(sheet)
    x, y, z = coords
    datasets = {}
    
    datasets['face'] = sheet.face_df.sort_values(z)
    datasets['vert'] = sheet.vert_df.sort_values(z)
    edge_z = 0.5 * (sheet.upcast_srce(sheet.vert_df[z]) +
                    sheet.upcast_trgt(sheet.vert_df[z]))
    datasets['edge'] = sheet.edge_df.copy()
    datasets['edge'][z] = edge_z
    datasets['edge'] = datasets['edge'].sort_values(z)
    
    tmp_sheet = Sheet('tmp', datasets,
                      sheet.specs)
    tmp_sheet.reset_index()

    draw_specs = {
        'vert': {
            'visible': False
            },
        'edge': {
            'color': tmp_sheet.edge_df[z],
            #'zorder': depth.values
            }
        }
    
    fig, ax = sheet_view(tmp_sheet, coords[:2], mode='2D', **draw_specs)
    ax.set_xlim(-15, 15)
    ax.set_ylim(-10, 10)
    ax.set_facecolor('#404040')
    ax.set_xticks([])
    ax.set_yticks([])
    fig.set_size_inches((10, 12))
    return fig, ax

fig, ax = leg_joint_view(sheet)


RuntimeError: Could not import backend "jupyter_rfb":
The jupyter_rfb backend relies on a the jupyter_rfb library: ``pip install jupyter_rfb``

### Choosing an apoptotic cell 

In [5]:
apoptotic_cell = 16
print('Apoptotic cell position:\n{}'.format(sheet.face_df.loc[apoptotic_cell, sheet.coords]))
apoptotic_edges = sheet.edge_df[sheet.edge_df['face'] == apoptotic_cell]
apoptotic_verts = apoptotic_edges['srce'].values
print("Indices of the apoptotic vertices: {}".format(apoptotic_verts))

Apoptotic cell position:
x   -7.994251
y    3.348658
z   -4.493606
Name: 16, dtype: float64
Indices of the apoptotic vertices: [33 42 34 41 43 44]


Cell behaviours, such as division or apoptosis, are defined as a series of unit changes in the tissue, such as the cell growth, or the abscission of the cell face once division is complete. For a given behavior, the next step often depends on the current state. This is managed by an `EventManager` object, which runs the behaviour.





In [6]:
from tyssue.behaviors.sheet import apoptosis
from tyssue.behaviors import EventManager


manager = EventManager('face')


sheet.settings['apoptosis'] = {
    'shrink_rate': 1.2,
    'critical_area': 8.,
    'radial_tension': 0.2,
    'contractile_increase': 0.3,
    'contract_span': 2
    }

sheet.face_df['id'] = sheet.face_df.index.values
manager.append(apoptosis, face_id=apoptotic_cell, **sheet.settings['apoptosis'])


Here is the code for the apoptosis behavior:

```py
def apoptosis(sheet, manager, **kwargs):
    """Apoptotic behavior

    While the cell's apical area is bigger than a threshold, the
    cell shrinks, and the contractility of its neighbors is increased.
    once the critical area is reached, the cell is eliminated
    from the apical surface through successive type 1 transition. Once
    only three sides are left, the cell is eliminated from the tissue.

    Parameters
    ----------
    sheet : a :class:`Sheet` object
    manager : a :class:`EventManager` object
    face_id : int,
        the id of the apoptotic cell
    shrink_rate : float, default 0.1
        the rate of reduction of the cell's prefered volume
        e.g. the prefered volume is devided by a factor 1+shrink_rate
    critical_area : area at which the face is eliminated from the sheet
    radial_tension : amount of radial tension added at each contraction steps
    contractile_increase : increase in contractility at the cell neighbors
    contract_span : number of neighbors affected by the contracitity increase
    geom : the geometry class used
    """

    apoptosis_spec = default_apoptosis_spec
    apoptosis_spec.update(**kwargs)
    face = apoptosis_spec["face"]

    if sheet.face_df.loc[face, "area"] > apoptosis_spec["critical_area"]:
        # Shrink and pull
        shrink(sheet, face, apoptosis_spec["shrink_rate"])
        ab_pull(sheet, face, apoptosis_spec["radial_tension"])
        # contract neighbors
        neighbors = sheet.get_neighborhood(
            face, apoptosis_spec["contract_span"]
        ).dropna()
        neighbors["id"] = sheet.face_df.loc[neighbors.face, "id"].values
        manager.extend(
            [
                (
                    contraction,
                    {
                        "face_id": neighbor["id"],
                        "contractile_increase": (
                            apoptosis_spec["contractile_increase"] / neighbor["order"],
                        ),
                    },
                )
                for _, neighbor in neighbors.iterrows()
            ]
        )
        done = False
    else:
        if sheet.face_df.loc[face, "num_sides"] > 3:
            exchange(sheet, face, apoptosis_spec["geom"])
            done = False
        else:
            remove(sheet, face, apoptosis_spec["geom"])
            done = True
    if not done:
        manager.append(apoptosis, **apoptosis_spec)
```


We perform the events at each time points to run the simulation

In [7]:
t = 0
stop=100

progress = widgets.IntProgress(min=0, max=stop)
progress.value = 0
display(progress)

history = History(sheet)

while manager.current and t < stop:
    manager.execute(sheet)
    t += 1
    progress.value = t
    res = solver.find_energy_min(sheet, geom, model, **min_settings)
    history.record()
    manager.update()
        

IntProgress(value=0)

### 3D view with napari

In [9]:
def sheet_napari(sheet, coords=None, interactive=True, **draw_specs_kw):
    """Uses VisPy to display an epithelium"""
    draw_specs = sheet_spec()
    spec_updater(draw_specs, draw_specs_kw)

    viewer = napari.Viewer()

    # Vispy scene creation
    # if coords is None:
    #     coords = ["x", "y", "z"]
    # canvas = scene.SceneCanvas(keys="interactive", show=True, size=(1240, 720))

    canvas = viewer.window.qt_viewer.canvas
    
    view = canvas.central_widget.add_view()
    view.camera = "turntable"
    view.camera.aspect = 1
    view.bgcolor = vp.color.Color("#222222")
    if draw_specs["edge"]["visible"]:
        wire = edge_visual(sheet, coords, **draw_specs["edge"])
        view.add(wire)
    if draw_specs["face"]["visible"]:
        mesh = face_visual(sheet, coords, **draw_specs["face"])
        view.add(mesh)

    canvas.show()
    view.camera.set_range()
    if interactive:
        napari.run()
    return canvas, view



# fig, mesh = sheet_napari(sheet, coords=['z', 'x', 'y'], edge={"color":color}, mode="3D", interactive=True)
fig, mesh = sheet_napari(sheet, coords=['z', 'x', 'y'], mode="3D", interactive=True)
fig




DEBUG : 2022-10-11 12:17:05,702 : MainThread : /Users/kharrington/miniconda3/envs/nesoi/lib/python3.9/site-packages/h5py/__init__.py:47 : Creating converter from 7 to 5
DEBUG : 2022-10-11 12:17:05,702 : MainThread : /Users/kharrington/miniconda3/envs/nesoi/lib/python3.9/site-packages/h5py/__init__.py:47 : Creating converter from 5 to 7
DEBUG : 2022-10-11 12:17:05,702 : MainThread : /Users/kharrington/miniconda3/envs/nesoi/lib/python3.9/site-packages/h5py/__init__.py:47 : Creating converter from 7 to 5
DEBUG : 2022-10-11 12:17:05,703 : MainThread : /Users/kharrington/miniconda3/envs/nesoi/lib/python3.9/site-packages/h5py/__init__.py:47 : Creating converter from 5 to 7
/var/folders/z0/_bcy172x6xg12j6wylfkhc_h0000gq/T/ipykernel_51919/2921983322.py:13: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_

<VispyCanvas (PyQt5) at 0x2aa7b60a0>

DEBUG : 2022-10-11 12:17:13,043 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/image/_image_slice.py:67 : ImageSlice.__init__
DEBUG : 2022-10-11 12:17:13,047 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1331 : Layer.refresh
DEBUG : 2022-10-11 12:17:13,056 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/_vispy/layers/base.py:137 : VispyBaseLayer._on_matrix_change
DEBUG : 2022-10-11 12:17:13,059 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/_vispy/layers/base.py:137 : VispyBaseLayer._on_matrix_change
DEBUG : 2022-10-11 12:17:13,061 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/_vispy/layers/base.py:137 : VispyBaseLayer._on_matrix_change
DEBUG : 2022-10-11 12:17:13,083 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/image/image.py:746 : Image._make_slice_request: ndim=2 ndisplay=2 last_used=0 range=((0.0, 

DEBUG : 2022-10-11 12:17:15,861 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:15,875 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:15,891 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:15,918 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  6

DEBUG : 2022-10-11 12:17:16,418 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:16,425 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:16,441 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  614.79821812])], (806, 1832)
DEBUG : 2022-10-11 12:17:16,458 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-561.16797057, -103.79824256]), array([1072.16790531,  6

...

DEBUG : 2022-10-11 12:17:27,121 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:27,135 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:27,169 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:27,185 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14

DEBUG : 2022-10-11 12:17:29,471 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:29,489 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:29,504 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:29,519 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14

DEBUG : 2022-10-11 12:17:29,976 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:29,988 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:30,000 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:30,016 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14

DEBUG : 2022-10-11 12:17:30,609 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:30,616 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:30,630 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:30,647 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14

DEBUG : 2022-10-11 12:17:33,618 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.5304600082884376, [array([ -13.97368444, -130.14442276]), array([524.97367717, 641.14441954])], (1454, 1016)
DEBUG : 2022-10-11 12:17:33,931 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-197.4119186 , -103.79824256]), array([708.41190776, 614.79821812])], (806, 1016)
DEBUG : 2022-10-11 12:17:33,946 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-197.4119186 , -103.79824256]), array([708.41190776, 614.79821812])], (806, 1016)
DEBUG : 2022-10-11 12:17:34,437 : MainThread : /Users/kharrington/git/kephale/nesoi/repos/napari/napari/layers/base/base.py:1701 : _update_draw: 0.8915589221191939, [array([-197.85769804, -103.79824256]), array([708.8576872 , 614.7982

In [15]:
draw_specs = sheet_spec()
mesh = face_visual(sheet, None, **draw_specs["face"])
dir(mesh)

['_Frozen__isfrozen',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_child',
 '_bounds',
 '_bounds_changed',
 '_build_color_transform',
 '_canvas',
 '_children',
 '_clim',
 '_clim_values',
 '_clip_children',
 '_clipper',
 '_clippers',
 '_cmap',
 '_color',
 '_compute_bounds',
 '_configure_gl_state',
 '_data_changed',
 '_describe_tree',
 '_document',
 '_document_node',
 '_draw_mode',
 '_ensure_vec4_func',
 '_filters',
 '_get_hook',
 '_hooks',
 '_id',
 '_index_buffer',
 '_interactive',
 '_meshdata',
 '_name',
 '_next_id',
 '_opacity',
 '_opacity_filter',
 '_order',
 '_parent',
 '_picking',
 '_picking_filter',
 '_prepare_draw',
 '_prepare_transforms',
 '_program',
 

In [ ]:

color = sheet.vert_df['y']
fig, mesh = sheet_view(sheet, coords=['z', 'x', 'y'], edge={"color":color}, mode="3D")
fig

In [16]:
browse_history(history, edge={"color":lambda s : s.edge_df["length"]})

NameError: name 'browse_history' is not defined

Provided you have ImageMagick installed, we can convert the output to a nice gif, like so:

In [ ]:
create_gif(
    history,
    'single_apopto.gif',
    num_frames=15,
    draw_func=leg_joint_view
)

In [ ]:
# Image("single_apopto.gif")

In [8]:
import pickle

pickle.dump(history, open( "apoptosis_history.pickle", "wb" ))